# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from contextlib import redirect_stdout
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from inflation import *
from regression import *
from utils import *

In [3]:
estimation_method = 'cloud'  # hardware-capex-energy, hardware-acquisition, cloud
compute_threshold_method = 'top_n'  # top_n, window_percentile
compute_threshold = 10  # e.g. 10 to select top 10; 75 to select top 25%
variant = 'original'  # whatever else distinguishes this run, e.g. 'excluding-AlphaGo'
exclude_models_containing = []  # ['GNMT', 'AlphaZero', 'AlphaGo Master', 'AlphaGo Zero']

estimation_method_lookup = {
    'hardware-capex-energy': estimate_hardware_capex_energy,
    'hardware-acquisition': estimate_hardware_acquisition_cost,
    'cloud': estimate_cloud_costs,
}
cost_estimation_function = estimation_method_lookup[estimation_method]

results_dir = f'results/{estimation_method}-{compute_threshold_method}={compute_threshold}-{variant}/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_threshold_method=compute_threshold_method, compute_threshold=compute_threshold,
)

In [5]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(87, 5702, 405)

# Cost estimation

In [6]:
with open(f'{results_dir}/cost_estimation.out', 'w') as f:
    with redirect_stdout(f):
        cost_df = cost_estimation_function(frontier_pcd_df, hardware_df, price_df)

In [7]:
if estimation_method == 'hardware-capex-energy':
    frontier_pcd_df_copy = frontier_pcd_df.copy()
    with open(f'{results_dir}/component_cost_estimation.out', 'w') as f:
        with redirect_stdout(f):
            component_cost_df = cost_estimation_function(frontier_pcd_df_copy, hardware_df, price_df, separate_components=True)

In [8]:
cost_df

,Model,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Hardware utilization (temp),BenchmarkHub-v1,Hugging Face developer id,Post-training compute (FLOP),Post-training compute notes,Hardware maker,benchmarks/models,Maybe over 1e25 FLOP,Updated dataset size,Cost
80,Doubao-pro,Language,"Language modeling/generation,Question answerin...",NaN,Training cost,NaN,API access,https://www.volcengine.com/docs/6360/1264663,NaN,Doubao General Model Pro (Doubao-pro),...,NaN,NaN,NaN,NaN,NaN,NaN,doubao-pro-32k,NaN,NaN,NaN
305,Grok-2,"Language,Vision,Multimodal","Chat,Language modeling/generation,Question ans...",NaN,Training cost,NaN,Hosted access (no API),https://x.ai/blog/grok-2,NaN,Grok-2 Beta Release,...,NaN,Grok-2 Beta,NaN,NaN,NaN,NVIDIA,"grok-2-1212,grok-2-vision-1212,grok-2-0813",NaN,NaN,3.402673e+07
336,Mistral Large 2,Language,"Language modeling/generation,Translation,Code ...","Albert Jiang, Alexandre Sablayrolles, Alexis T...",Training cost,likely high training cost since previous Mistr...,Open weights (non-commercial),https://mistral.ai/news/mistral-large-2407/,NaN,"Top-tier reasoning for high-complexity tasks, ...",...,NaN,"Mistral Large 2,Mistral Large 2,Mistral Large ...",mistralai,NaN,NaN,NaN,"Mistral-Large-Instruct-2407,Mistral-Large-Inst...",NaN,NaN,NaN
339,Llama 3.1-405B,Language,Language modeling/generation,"Abhimanyu Dubey, Abhinav Jauhri, Abhinav Pande...","SOTA improvement,Training cost","High training compute, exceeds 4o and Claude 3...",Open weights (restricted use),https://ai.meta.com/research/publications/the-...,NaN,The Llama 3 Herd of Models,...,NaN,"Llama 3.1-405B,Llama 3.1-405B,Llama 3.1-405B,L...",meta-llama,9.400000e+22,Section 4 gives detail about the post-training...,NVIDIA,Llama-3.1-405B-Instruct,NaN,NaN,3.072877e+07
377,Claude 3.5 Sonnet,"Multimodal,Language,Vision","Chat,Image captioning,Code generation,Language...",NaN,"Significant use,SOTA improvement","""It also sets new performance standards in eva...",API access,https://www-cdn.anthropic.com/fed9cc193a14b841...,NaN,Claude 3.5 Sonnet,...,NaN,"Claude 3.5 Sonnet (2024-06-20),Claude 3.5 Sonn...",NaN,NaN,NaN,NaN,"claude-3-5-sonnet-20240620,claude-3-5-sonnet-2...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,Unreleased,https://www.nature.com/articles/nature16961,16057.0,Mastering the game of Go with deep neural netw...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 checked out of 1,NaN
2015,ResNet-152 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.03385,175697.0,Deep Residual Learning for Image Recognition,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 checked out of 1,NaN
2017,ResNet-101 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,Open weights (unrestricted),https://arxiv.org/abs/1512.03385,175697.0,Deep Residual Learning for Image Recognition,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2853.0,Deep Speech 2: End-to-End Speech Recognition i...,...,NaN,NaN,NaN,NaN,NaN,NVIDIA,NaN,NaN,0 checked out of 1,NaN


In [9]:
cost_df['Cost'].notna().sum()

44

In [10]:
cost_df.dropna(subset=['Cost'])['Training time (hours)'].notna().sum()

30

In [11]:
cost_df.dropna(subset=['Cost'])['Hardware utilization'].notna().sum()

19

Exclusion

In [12]:
cost_df[['Model', 'Publication date']].tail(15)

,Model,Publication date
1877,Libratus,2017-08-19
1886,OpenAI TI7 DOTA 1v1,2017-08-11
1892,ConvS2S (ensemble of 8 models),2017-07-25
1898,JFT,2017-07-10
1922,MoE-Multi,2017-01-23
1942,PolyNet,2016-11-17
1948,NASv3 (CIFAR-10),2016-11-05
1957,Xception,2016-10-07
1958,GNMT,2016-09-26
1970,ResNet-200,2016-09-17


In [13]:
for kw in exclude_models_containing:
    cost_df = cost_df[cost_df['Model'].str.contains(kw) == False]
cost_df[['Model', 'Publication date']].tail(15)

,Model,Publication date
1877,Libratus,2017-08-19
1886,OpenAI TI7 DOTA 1v1,2017-08-11
1892,ConvS2S (ensemble of 8 models),2017-07-25
1898,JFT,2017-07-10
1922,MoE-Multi,2017-01-23
1942,PolyNet,2016-11-17
1948,NASv3 (CIFAR-10),2016-11-05
1957,Xception,2016-10-07
1958,GNMT,2016-09-26
1970,ResNet-200,2016-09-17


Use the below to check data availability for specific systems

In [14]:
# system = 'WizardLM-7B'
# row = cost_df.loc[cost_df['Model'] == system]
# print('Cost:', row['Cost'].values[0])
# print('Training hardware:', row['Training hardware'].values[0])
# print('Training time (hours):', row['Training time (hours)'].values[0])
# print('Hardware quantity:', row['Hardware quantity'].values[0])
# print('Hardware utilization:', row['Hardware utilization'].values[0])

# Apply inflation adjustment

In [15]:
cost_df['Cost'].dropna()

305     3.402673e+07
339     3.072877e+07
386     1.183531e+07
556     6.555406e+06
582     7.341321e+06
588     8.980070e+06
694     1.914000e+08
717     6.561306e+06
802     2.298470e+07
824     2.565734e+07
969     2.866801e+07
1034    7.752000e+07
1110    8.324027e+06
1207    1.547798e+06
1246    1.462579e+06
1257    1.218724e+07
1280    1.276416e+06
1313    2.028237e+06
1317    3.391488e+06
1360    6.209280e+06
1368    9.550237e+05
1413    4.390244e+05
1422    3.994580e+05
1462    2.433024e+05
1479    5.971968e+05
1482    3.425778e+05
1547    9.289728e+05
1556    2.268444e+05
1563    4.120320e+06
1604    6.635520e+05
1626    1.327104e+05
1630    1.488569e+05
1632    3.568435e+05
1635    4.769648e+04
1636    2.171250e+05
1650    2.355302e+05
1651    2.025830e+05
1677    1.486848e+05
1698    3.280458e+04
1703    2.433024e+04
1769    1.081344e+04
1898    3.096000e+04
1957    1.857600e+04
1958    2.819639e+05
Name: Cost, dtype: float64

In [16]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2024-12-01')

In [17]:
cost_df['Cost (inflation-adjusted)'].dropna()

305     3.390939e+07
339     3.059700e+07
386     1.180613e+07
556     6.560959e+06
582     7.330879e+06
588     8.967297e+06
694     1.919146e+08
717     6.594291e+06
802     2.318910e+07
824     2.588551e+07
969     2.904054e+07
1034    7.856202e+07
1110    8.514143e+06
1207    1.581238e+06
1246    1.489702e+06
1257    1.242226e+07
1280    1.323122e+06
1313    2.098625e+06
1317    3.509187e+06
1360    6.422820e+06
1368    9.896170e+05
1413    4.552313e+05
1422    4.142043e+05
1462    2.520651e+05
1479    6.214023e+05
1482    3.564631e+05
1547    9.717088e+05
1556    2.372801e+05
1563    4.336473e+06
1604    6.989785e+05
1626    1.417982e+05
1630    1.590503e+05
1632    3.812795e+05
1635    5.096264e+04
1636    2.319933e+05
1650    2.518844e+05
1651    2.166495e+05
1677    1.604465e+05
1698    3.546371e+04
1703    2.635018e+04
1769    1.167940e+04
1898    3.353035e+04
1957    2.033970e+04
1958    3.087349e+05
Name: Cost (inflation-adjusted), dtype: float64

In [18]:
# Equal number of non-null values
assert cost_df['Cost (inflation-adjusted)'].notna().sum() == cost_df['Cost'].notna().sum()

# Regression

In [19]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [20]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.758
Model:                            OLS   Adj. R-squared:                  0.752
Method:                 Least Squares   F-statistic:                     131.6
Date:                Thu, 27 Feb 2025   Prob (F-statistic):           1.59e-14
Time:                        12:34:01   Log-Likelihood:                -32.567
No. Observations:                  44   AIC:                             69.13
Df Residuals:                      42   BIC:                             72.70
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -870.0344     76.364    -11.393      0.000   -1024.144    -715.925
x1             0.4334      0.038     11.473      0.000       0.357       0.510
==============================================================================
Omnibus:                        6.093   Durbin-Watson:                   1.433
Prob(Omnibus):                  0.048   Jarque-Bera (JB):                5.043
Skew:                           0.802   Prob(JB):                       0.0803
Kurtosis:                       3.419   Cond. No.                     1.97e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.97e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
with open(f'{results_dir}/regression_results.out', 'w') as f:
    with redirect_stdout(f):
        print_growth_rates(reg_results, round_digits=None)
print_growth_rates(reg_results, ci=90, round_digits=5)

N=44.0
R^2=0.76
0.43342 OOMs/year (90% CI: 0.36988, 0.49696)
2.7128x/year (90% CI: 2.34357x, 3.14021x)
doubling time of 8.33458 months (90% CI: 7.26893, 9.76636)


In [22]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [23]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,3.303604,0.253978,2.876426,3.730783,2.331495,4.275714,2015.00000,2015-01-01
1,3.347384,0.250350,2.926307,3.768461,2.377940,4.316828,2015.10101,2015-02-06
2,3.391164,0.246729,2.976178,3.806149,2.424350,4.357977,2015.20202,2015-03-15
3,3.434943,0.243113,3.026039,3.843847,2.470725,4.399162,2015.30303,2015-04-21
4,3.478723,0.239503,3.075890,3.881555,2.517063,4.440382,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,7.462669,0.144022,7.220432,7.704907,6.556472,8.368867,2024.59596,2024-08-06
96,7.506449,0.147240,7.258799,7.754099,6.598790,8.414108,2024.69697,2024-09-12
97,7.550229,0.150485,7.297119,7.803338,6.641065,8.459393,2024.79798,2024-10-19
98,7.594008,0.153757,7.335396,7.852621,6.683297,8.504720,2024.89899,2024-11-25


In [24]:
predicted_cost_df.to_csv(results_dir + 'predicted_cost_dataset.csv', index=False)

# Export data

In [25]:
keep_cols = [
    'Model',
    'Domain',
    'Task',
    'Model accessibility',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    # 'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,Model,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Training hardware,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted)
80,Doubao-pro,Language,"Language modeling/generation,Question answerin...",API access,Doubao General Model Pro (Doubao-pro),2024-10-28,ByteDance,5.000000e+11,2.505000e+25,8.350000e+12,...,NaN,China,NaN,NaN,NaN,NaN,NaN,"There is no paper to reference, also no inform...",NaN,NaN
305,Grok-2,"Language,Vision,Multimodal","Chat,Language modeling/generation,Question ans...",Hosted access (no API),Grok-2 Beta Release,2024-08-13,xAI,NaN,2.960000e+25,NaN,...,NVIDIA H100 SXM5 80GB,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,3.402673e+07,3.390939e+07
336,Mistral Large 2,Language,"Language modeling/generation,Translation,Code ...",Open weights (non-commercial),"Top-tier reasoning for high-complexity tasks, ...",2024-07-24,Mistral AI,1.230000e+11,2.130000e+25,NaN,...,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339,Llama 3.1-405B,Language,Language modeling/generation,Open weights (restricted use),The Llama 3 Herd of Models,2024-07-23,Meta AI,4.050000e+11,3.800000e+25,1.560000e+13,...,NVIDIA H100 SXM5 80GB,United States of America,NaN,NaN,16384.0,0.4042,NaN,NaN,3.072877e+07,3.059700e+07
377,Claude 3.5 Sonnet,"Multimodal,Language,Vision","Chat,Image captioning,Code generation,Language...",API access,Claude 3.5 Sonnet,2024-06-20,Anthropic,NaN,3.650001e+25,NaN,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,AlphaGo Lee,Games,Go,Unreleased,Mastering the game of Go with deep neural netw...,2016-01-27,DeepMind,NaN,1.900000e+21,2.940000e+07,...,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,ResNet-152 (ImageNet),Vision,Image classification,NaN,Deep Residual Learning for Image Recognition,2015-12-10,Microsoft,6.020000e+07,1.041408e+19,1.280000e+06,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017,ResNet-101 (ImageNet),Vision,Image classification,Open weights (unrestricted),Deep Residual Learning for Image Recognition,2015-12-10,Microsoft,4.450000e+07,7.004000e+18,1.280000e+06,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018,DeepSpeech2 (English),Speech,Speech recognition,NaN,Deep Speech 2: End-to-End Speech Recognition i...,2015-12-08,Baidu Research - Silicon Valley AI Lab,3.800000e+07,2.600000e+19,1.633392e+08,...,NVIDIA GeForce GTX TITAN X,United States of America,NaN,NaN,16.0,0.4484,NaN,NaN,NaN,NaN


In [26]:
cost_df[keep_cols].to_csv(results_dir + 'cost_dataset.csv', index=False)

# Plots

In [27]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='Model',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2024 USD, log scale)')

# title
fig.update_layout(title_text=get_cost_plot_title(estimation_method, compute_threshold_method, compute_threshold))

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    )
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [28]:
label_systems = ['GNMT', 'AlphaGo Zero', 'DALL-E', 'GPT-3 175B (davinci)', 'GPT-4', 'Llama 3.1-405B', 'Grok-2']

tpu_mask = cost_df['Training hardware'].str.contains('TPU', na=False)
tpu_cost_df = cost_df.loc[tpu_mask]
gpu_cost_df = cost_df.loc[~tpu_mask]

fig = px.scatter(
    gpu_cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    log_y=True,
)
fig.add_scatter(
    x=tpu_cost_df['Publication date'],
    y=tpu_cost_df['Cost (inflation-adjusted)'],
    mode='markers',
    marker_symbol='circle-open' if estimation_method != 'cloud' else 'circle',
    name='Using estimated cost of TPU' if estimation_method != 'cloud' else '',
    showlegend=estimation_method != 'cloud',
)
fig.add_scatter(
    x=gpu_cost_df.loc[gpu_cost_df['Model'].isin(label_systems)]['Publication date'],
    y=gpu_cost_df.loc[gpu_cost_df['Model'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=gpu_cost_df.loc[gpu_cost_df['Model'].isin(label_systems)]['Model'],
    mode='text',
    showlegend=False,
)
fig.add_scatter(
    x=tpu_cost_df.loc[tpu_cost_df['Model'].isin(label_systems)]['Publication date'],
    y=tpu_cost_df.loc[tpu_cost_df['Model'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=tpu_cost_df.loc[tpu_cost_df['Model'].isin(label_systems)]['Model'],
    mode='text',
    showlegend=False,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='90% CI of mean',
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# axis limits
# fig.update_xaxes(range=[pred_start_date, pred_end_date])
fig.update_xaxes(range=['2015-01-01', '2025-06-01'])  # manual
if estimation_method == 'hardware-acquisition':
    fig.update_yaxes(range=[4, 10])
else:
    fig.update_yaxes(range=[1, 9])

# legend on bottom-right of the axes
fig.update_layout(
    legend=dict(
        x=0.45,
        y=0.05,
    )
)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2024 USD, log scale)')

# title
fig.update_layout(title_text=get_cost_plot_title(estimation_method, compute_threshold_method, compute_threshold))

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    ),
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=60, b=10))

save_plot(fig, results_dir, 'cost_regression')

fig.show()

# Cost components

In [29]:
cost_component_names = [
    'AI accelerator chip cost',
    'Other server components cost',
    'Cluster-level interconnect cost',
    'Energy cost',
]

In [30]:
for key in cost_component_names:
    component_cost_df[f"{key} (%)"] = component_cost_df[key] / component_cost_df['Cost'] * 100
component_cost_df['AI accelerator chip cost (%)']

NameError: name 'component_cost_df' is not defined

In [31]:
cost_component_pc_names = [name + ' (%)' for name in cost_component_names]
filtered_component_cost_df = component_cost_df.dropna(subset=cost_component_pc_names).sort_values(by='Publication date')

In [ ]:
# Stacked bar chart of cost components, using component_cost_df
fig = px.bar(
    filtered_component_cost_df,
    x='Model',
    y=cost_component_pc_names,
    barmode='stack',
)

# axis labels
fig.update_xaxes(title_text='ML model')
fig.update_yaxes(title_text='% of amortized hardware CapEx + energy')
fig.update_layout(
    legend=dict(
        title_text='Cost component',
        x=0.60,
        y=0.05,
    )
)
# limits 0 to 100
fig.update_yaxes(range=[0, 100])

fig.update_yaxes(tickvals=list(range(0, 101, 10)))

# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_component_percentage')

fig.show()

In [ ]:
filtered_component_cost_df.head()

,Model,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Updated dataset size,Cost,AI accelerator chip cost,Other server components cost,Cluster-level interconnect cost,Energy cost,AI accelerator chip cost (%),Other server components cost (%),Cluster-level interconnect cost (%),Energy cost (%)
2018,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2853.0,Deep Speech 2: End-to-End Speech Recognition i...,...,0 checked out of 1,185.456639,63.953273,40.930095,24.602271,55.971000,34.484219,22.069900,13.265781,30.180101
1958,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,Hosted access (no API),https://arxiv.org/abs/1609.08144,6483.0,Google's Neural Machine Translation System: Br...,...,0 checked out of 1,177459.232941,77894.044829,49852.188691,29965.165887,19747.833534,43.894050,28.092192,16.885662,11.128096
1957,Xception,Vision,Image classification,François Chollet,Highly cited,NaN,NaN,https://arxiv.org/abs/1610.02357,13038.0,Xception: Deep Learning with Depthwise Separab...,...,0 checked out of 1,11554.506253,5064.230483,3241.107509,1948.165702,1301.002560,43.829051,28.050593,16.860657,11.259698
1942,PolyNet,Vision,Image classification,"X Zhang, Z Li, C Change Loy",SOTA improvement,"""The Very Deep PolyNet, designed following thi...",NaN,https://arxiv.org/abs/1611.05725,282.0,PolyNet: A Pursuit of Structural Diversity in ...,...,0 checked out of 1,563.599706,178.564122,114.281038,68.692074,202.062472,31.682792,20.276987,12.188096,35.852125
1922,MoE-Multi,Language,"Language modeling,Translation","N Shazeer, A Mirhoseini, K Maziarz, A Davis","Highly cited,SOTA improvement","""On large language modeling and machine transl...",Unreleased,https://arxiv.org/abs/1701.06538,2037.0,Outrageously Large Neural Networks: The Sparse...,...,1 checked out of 1,3538.189418,1519.646471,972.573741,584.594865,461.374341,42.949834,27.487894,16.522430,13.039843


In [34]:
filtered_component_cost_df.to_csv(results_dir + 'cost_components.csv', index=False)

In [ ]:
# Average percentage for each component
filtered_component_cost_df[cost_component_pc_names].mean()

AI accelerator chip cost (%)           44.813469
Other server components cost (%)       29.152130
Cluster-level interconnect cost (%)    17.349955
Energy cost (%)                         8.684446
dtype: float64

In [ ]:
fig = px.bar(
    filtered_component_cost_df,
    x='Model',
    y='Energy cost (%)',
    barmode='stack',
    # labels='Cost %',
    # text='Energy cost %',
)
# axis labels
fig.update_xaxes(title_text='Model')
fig.update_yaxes(title_text='Energy cost (% of amortized hardware CapEx + energy)')
# fig.update_layout(
#     legend=dict(
#         title_text='Cost component',
#         x=0.75,
#         y=0.05,
#     )
# )
# limits 0 to 100
fig.update_yaxes(range=[0, 30])
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_percentage')

fig.show()

In [ ]:
fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y='Energy cost',
    text='Model',
)
# axis labels
fig.update_xaxes(title_text='Model')
fig.update_yaxes(title_text='Energy cost')
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_cost')

fig.show()

In [ ]:
from energy import energy_price

# Stacked bar chart of cost components, using component_cost_df
filtered_component_cost_df.loc[:, 'Energy (kWh)'] = [
    row['Energy cost'] / energy_price(row['Publication date'].year) 
    for _, row in filtered_component_cost_df.iterrows()
]
fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y='Energy (kWh)',
    text='Model',
)
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_kwh')

fig.show()

In [ ]:
filtered_component_cost_df.columns

Index(['Model', 'Domain', 'Task', 'Authors', 'Notability criteria',
       'Notability criteria notes', 'Model accessibility', 'Link', 'Citations',
       'Reference',
       ...
       'Cost', 'AI accelerator chip cost', 'Other server components cost',
       'Cluster-level interconnect cost', 'Energy cost',
       'AI accelerator chip cost (%)', 'Other server components cost (%)',
       'Cluster-level interconnect cost (%)', 'Energy cost (%)',
       'Energy (kWh)'],
      dtype='object', length=105)

In [ ]:
filtered_component_cost_df = filtered_component_cost_df.dropna(subset=['Training hardware'])
power_col = 'Power capacity for final training run (kW)'
filtered_component_cost_df.loc[:, power_col] = [
    cluster_power_capacity(row['Training hardware'], row['Hardware quantity'], hardware_df, row['Organization'])
    for _, row in filtered_component_cost_df.iterrows()
]

fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y=power_col,
    text='Model',
)
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'power_capacity_kw')

fig.show()

In [41]:
filtered_component_cost_df['Publication date (float)'] = datetime_to_float_year(
    pd.to_datetime(filtered_component_cost_df['Publication date'])
)

In [ ]:
power_reg_results = fit_ols_regression(
    filtered_component_cost_df,
    ['Publication date (float)'],
    power_col,
    logy=True
)
power_reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.868
Model:                            OLS   Adj. R-squared:                  0.865
Method:                 Least Squares   F-statistic:                     224.5
Date:                Fri, 21 Feb 2025   Prob (F-statistic):           1.53e-16
Time:                        17:55:02   Log-Likelihood:                -10.058
No. Observations:                  36   AIC:                             24.12
Df Residuals:                      34   BIC:                             27.28
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -727.4362     48.742    -14.924      0.000    -826.492    -628.380
x1             0.3614      0.024     14.983      0.000       0.312       0.410
==============================================================================
Omnibus:                        6.641   Durbin-Watson:                   1.917
Prob(Omnibus):                  0.036   Jarque-Bera (JB):                5.920
Skew:                           0.596   Prob(JB):                       0.0518
Kurtosis:                       4.589   Cond. No.                     1.80e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.8e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
with open(f'{results_dir}/power_regression_results.out', 'w') as f:
    with redirect_stdout(f):
        print_growth_rates(power_reg_results)
print_growth_rates(power_reg_results)

N=36.0
R^2=0.87
0.361389814255957 OOMs/year (90% CI: 0.32060602178535264, 0.4021736067265614)
2.2982105515758104x/year (90% CI: 2.092213604699331x, 2.524489721083439x)
doubling time of 9.99574366921502 months (90% CI: 8.982091036182354, 11.267286646244804)


In [ ]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [ ]:
predicted_power_df = get_predictions(power_reg_results, pred_years, ['Publication date (float)'])
predicted_power_df['Publication date'] = predicted_power_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_power_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,0.764251,0.152120,0.507028,1.021473,0.150980,1.377521,2015.00000,2015-01-01
1,0.800755,0.149850,0.547370,1.054140,0.189084,1.412425,2015.10101,2015-02-06
2,0.837259,0.147586,0.587703,1.086815,0.227164,1.447353,2015.20202,2015-03-15
3,0.873763,0.145327,0.628026,1.119499,0.265221,1.482305,2015.30303,2015-04-21
4,0.910267,0.143074,0.668340,1.152194,0.303253,1.517280,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,4.232133,0.105042,4.054515,4.409750,3.647766,4.816500,2024.59596,2024-08-06
96,4.268637,0.107127,4.087494,4.449780,3.683188,4.854085,2024.69697,2024-09-12
97,4.305141,0.109226,4.120447,4.489834,3.718584,4.891697,2024.79798,2024-10-19
98,4.341645,0.111339,4.153378,4.529911,3.753953,4.929336,2024.89899,2024-11-25


## Power plot

In [ ]:
label_systems = ['GNMT', 'AlphaGo Master', 'AlphaGo Zero', 'AlphaZero', 'DALL-E', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini 1.0 Ultra', 'Inflection-2']

fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y=power_col,
    log_y=True,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

fig.add_scatter(
    x=filtered_component_cost_df.loc[filtered_component_cost_df['Model'].isin(label_systems)]['Publication date'],
    y=filtered_component_cost_df.loc[filtered_component_cost_df['Model'].isin(label_systems)][power_col],
    text=filtered_component_cost_df.loc[filtered_component_cost_df['Model'].isin(label_systems)]['Model'],
    mode='text',
    showlegend=False,
)

# Shade in CI
fig.add_scatter(
    x=predicted_power_df['Publication date'],
    y=10**predicted_power_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_power_df['Publication date'],
    y=10**predicted_power_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='90% CI of mean',
)
fig.add_scatter(
    x=predicted_power_df['Publication date'],
    y=10**predicted_power_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**power_reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])
# fig.update_xaxes(range=['2015-01-01', '2025-01-01'])  # manual
# fig.update_yaxes(range=[1, 6])

# legend on bottom-right of the axes
fig.update_layout(
    legend=dict(
        x=0.45,
        y=0.05,
    )
)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Power (kW, log scale)')

# title
fig.update_layout(title_text='Cluster power required for final training run')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    ),
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=60, b=10))

save_plot(fig, results_dir, 'power_regression')

fig.show()